In [ ]:
# A better way to read the same file, handling BOM and end-of-line truncation

import re
import pandas as pd
from pathlib import Path

path = "arrets_ben.csv"
out_parquet = "arrets_ben.parquet"

# --- 1) Read header + dash line, remove BOM automatically ---
with open(path, encoding="utf-8-sig") as f:
    header_line = f.readline().rstrip("\n")
    dash_line   = f.readline().rstrip("\n")

# --- 2) Infer column spans from runs of dashes ---
colspecs = [(m.start(), m.end()) for m in re.finditer(r"-+", dash_line)]

# Make the last column go to end-of-line to avoid truncation
colspecs[-1] = (colspecs[-1][0], None)

# --- 3) Slice column names from the (BOM-stripped) header ---
raw_names = [header_line[s:] if e is None else header_line[s:e] for s, e in colspecs]
names = []
seen = {}
for nm in map(str.strip, raw_names):
    seen[nm] = seen.get(nm, -1) + 1
    names.append(nm if seen[nm] == 0 else f"{nm}_{seen[nm]}")

print("Detected columns:", len(names))
print(names[:10], "...")

# --- 4) Read the data as fixed-width (skip header + dashes) ---
df = pd.read_fwf(
    path,
    colspecs=colspecs,
    names=names,
    skiprows=2,
    na_values=["NULL"],
    encoding="utf-8-sig",
)
print(df.shape)
print(df.head(3))

In [ ]:
oct01 = df[df['DateCourse'] == '2024-10-01']

In [ ]:
oct01.to_csv('Oct-01.csv', index=False)

In [ ]:
import geopandas as gpd
stops = pd.read_csv('stops_df.csv')
lines = gpd.read_file("TPG_LIGNES-SHP/TPG_LIGNES.shp")
gstops = gpd.GeoDataFrame(
    stops,
    geometry=gpd.points_from_xy(stops["X"], stops["Y"]),
    crs="EPSG:21781"
)
line18 = lines[lines['LIGNE'] == '18']
print(lines.crs)

EPSG:2056


In [ ]:
import pandas as pd
import geopandas as gpd
import folium

# --- 0) Load data (your code) ---
stops = pd.read_csv("stops_df.csv", encoding="utf-8-sig")
lines = gpd.read_file("TPG_LIGNES-SHP/TPG_LIGNES.shp")        # EPSG:2056

# stops are LV03 (EPSG:21781) per your values
gstops_lv03 = gpd.GeoDataFrame(
    stops, geometry=gpd.points_from_xy(stops["X"], stops["Y"]), crs="EPSG:21781"
)

# Prepare line 18
line18 = lines[lines["LIGNE"] == "18"].copy()                  # EPSG:2056

# --- 1) Filter stops to ONLY those on line 18 ---

## Option A (preferred if you already have the ops dataframe `df`)
## uses CodeLong ↔ MNLP and C_Ligne == 18
def stops_for_line_via_ops(gstops_lv03, df, line_num=18):
    codes = (
        df.loc[df["C_Ligne"] == line_num, "CodeLong"]
          .astype(str).str.strip().unique()
    )
    return gstops_lv03[gstops_lv03["MNLP"].astype(str).str.strip().isin(codes)].copy()

# If you DON'T have `df` available in this notebook, use spatial proximity:
## Option B (spatial): keep stops within 60 m of the line geometry
def stops_for_line_via_spatial(gstops_lv03, line18, tol_m=60):
    # work in EPSG:2056 (meters) for spatial filtering
    gstops_2056 = gstops_lv03.to_crs(2056)
    # unify line18 geometry & buffer
    line18_union = line18.unary_union
    line18_buf = gpd.GeoSeries([line18_union], crs=2056).buffer(tol_m).iloc[0]
    # keep points within buffer
    mask = gstops_2056.geometry.within(line18_buf)
    return gstops_2056.loc[mask].copy()       # returns EPSG:2056

# Choose the method:
try:
    # if df is present, use ops-based filter
    stops18_2056 = stops_for_line_via_ops(gstops_lv03, df, line_num=18).to_crs(2056)
except NameError:
    # fallback to spatial method
    stops18_2056 = stops_for_line_via_spatial(gstops_lv03, line18, tol_m=60)

# --- 2) Convert to WGS84 for Folium ---
line18_wgs  = line18.to_crs(4326)
stops18_wgs = stops18_2056.to_crs(4326)

# --- 3) Build the map ---
minx, miny, maxx, maxy = line18_wgs.total_bounds
center = [(miny + maxy) / 2, (minx + maxx) / 2]
m = folium.Map(location=center, zoom_start=12, control_scale=True, tiles="cartodbpositron")

# Line layer (blue)
for _, row in line18_wgs.explode(index_parts=False).iterrows():
    folium.GeoJson(
        row.geometry,
        name="Ligne 18",
        style_function=lambda x: {"color": "#1565C0", "weight": 4, "opacity": 0.9},
        tooltip=row.get("NOM_LIGNE", "Ligne 18"),
    ).add_to(m)

# Stops layer (red)
stops_fg = folium.FeatureGroup(name="Stops (Ligne 18)")
for _, r in stops18_wgs.iterrows():
    folium.CircleMarker(
        location=(r.geometry.y, r.geometry.x),
        radius=4,
        color="#B71C1C",
        fill=True,
        fill_color="#E53935",
        fill_opacity=0.95,
        weight=1,
        tooltip=r.get("LIBP", r.get("MNLP", "")),
        popup=folium.Popup(
            f"<b>Stop:</b> {r.get('LIBP','')}"
            f"<br><b>CodeLong/MNLP:</b> {r.get('MNLP','')}"
            f"<br><b>ARR:</b> {r.get('ARR','')}",
            max_width=260,
        ),
    ).add_to(stops_fg)
stops_fg.add_to(m)

# Finish
m.fit_bounds([[miny, minx], [maxy, maxx]])
folium.LayerControl(collapsed=False).add_to(m)

# Save or display
m  # in notebook
# m.save("ligne18_stops_map.html")

In [ ]:
import folium

# convert stops to WGS84
gstops_wgs = gstops.to_crs(4326)

# map centered on all stops
minx, miny, maxx, maxy = gstops_wgs.total_bounds
center = [(miny+maxy)/2, (minx+maxx)/2]
m = folium.Map(location=center, zoom_start=12, tiles="cartodbpositron")

# plot stops as circle markers
for _, r in gstops_wgs.iterrows():
    lat, lon = r.geometry.y, r.geometry.x
    folium.CircleMarker(
        location=(lat, lon),
        radius=3,
        weight=1,
        fill=True,
        fill_opacity=0.9,
        tooltip=r.get("LIBP", r.get("MNLP", "")),  # stop name if available
    ).add_to(m)

# fit bounds
m.fit_bounds([[miny, minx], [maxy, maxx]])
m